In [ ]:
# Setup environment
!pip install kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download and unzip dataset
!kaggle datasets download -d grassknoted/asl-alphabet
!unzip asl-alphabet.zip



asl-alphabet.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  asl-alphabet.zip
replace asl_alphabet_test/asl_alphabet_test/A_test.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
!pip install pillow numpy
from PIL import Image
import os
import numpy as np

Images_train = []
Y_train = []

Images_val = []
Y_val = []

Images_test = []
Y_test = []

DIRECTORY = '/content/asl_alphabet_train/asl_alphabet_train'
Cnt = 0
div = 0

for FILENAME in os.listdir(DIRECTORY):
    directory = os.path.join(DIRECTORY, FILENAME)
    # Check if the current entry is a file
    for filename in os.listdir(directory):
         filepath = os.path.join(directory, filename)
         if os.path.isfile(filepath):
              image = Image.open(filepath)
              image = image.resize((150, 150))
              image = image.convert('L')
              image_array = np.array(image)
              image_array = image_array/255
              image_array = image_array[:, :, np.newaxis]
              if div <= 2500:
                Images_train.append(image_array)
                Y_train.append(Cnt)
              elif div > 2500 and div <= 2900:
                Images_val.append(image_array)
                Y_val.append(Cnt)
              else:
                Images_test.append(image_array)
                Y_test.append(Cnt)
              div += 1
              image.close()
    div = 0
    Cnt += 1
    print(f"Completed: {FILENAME}")

X_train = np.stack(Images_train)
Y_train = np.array(Y_train)

X_val = np.stack(Images_val)
Y_val = np.array(Y_val)

X_test = np.stack(Images_test)
Y_test = np.array(Y_test)

print(f"X_Train: {X_train.shape} , Y_Train: {Y_train.shape}")
print(f"X_val: {X_val.shape} , Y_val: {Y_val.shape}")
print(f"X_test: {X_test.shape} , Y_test: {Y_test.shape}")




Completed: V
Completed: space
Completed: N
Completed: nothing
Completed: A
Completed: G
Completed: P
Completed: Q
Completed: J
Completed: T
Completed: W
Completed: Y
Completed: K
Completed: B
Completed: L
Completed: X
Completed: U
Completed: S
Completed: E
Completed: I
Completed: D
Completed: H
Completed: del
Completed: R
Completed: O
Completed: F
Completed: M
Completed: Z
Completed: C
X_Train: (72529, 150, 150, 1) , Y_Train: (72529,)
X_val: (11600, 150, 150, 1) , Y_val: (11600,)
X_test: (2871, 150, 150, 1) , Y_test: (2871,)


In [ ]:
from keras.utils import to_categorical

#Covert to 32-bit floating point
X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_test = X_test.astype('float32')

#Flatten Data
#X_train_flatten = X_train.reshape(X_train.shape[0], X_train.shape[1] * X_train.shape[2])
#X_val_flatten = X_val.reshape(X_val.shape[0], X_val.shape[1] * X_val.shape[2])
#X_test_flatten = X_test.reshape(X_test.shape[0], X_test.shape[1] * X_test.shape[2])

#print(f"X_Train: {X_train_flatten.shape} , Y_Train: {Y_train.shape}")
#print(f"X_val: {X_val_flatten.shape} , Y_val: {Y_val.shape}")
#print(f"X_test: {X_test_flatten.shape} , Y_test: {Y_test.shape}")

Y_train_class = to_categorical(Y_train, 29)
Y_val_class = to_categorical(Y_val, 29)
Y_test_class = to_categorical(Y_test, 29)

print(Y_train_class.shape)
#print(Y_val_class)
#print(Y_test_class)

(72529, 29)


In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.datasets import cifar10
from keras.optimizers import legacy
from keras.layers import Conv2D, MaxPooling2D, Flatten, LSTM

model = Sequential()
#model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(200, 200)))
#model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu', input_shape=(150, 150)))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(29, activation='softmax'))

# Let's train the model using RMSprop
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

model.fit(X_train, Y_train_class, batch_size=64, epochs=10, validation_data=(X_val, Y_val_class))

model.summary()

score = model.evaluate(X_test, Y_test_class, verbose=0)

print(score[0])

Epoch 1/10
1134/1134 [==============================] - 140s 121ms/step - loss: 6.8292 - categorical_accuracy: 0.0381 - val_loss: 3.3729 - val_categorical_accuracy: 0.0355
Epoch 2/10
1134/1134 [==============================] - 134s 118ms/step - loss: 3.3677 - categorical_accuracy: 0.0349 - val_loss: 3.3662 - val_categorical_accuracy: 0.0365
Epoch 3/10
1134/1134 [==============================] - 133s 118ms/step - loss: 3.3635 - categorical_accuracy: 0.0340 - val_loss: 3.3614 - val_categorical_accuracy: 0.0365
Epoch 4/10
1134/1134 [==============================] - 134s 118ms/step - loss: 3.3619 - categorical_accuracy: 0.0352 - val_loss: 3.3623 - val_categorical_accuracy: 0.0363
Epoch 5/10
1134/1134 [==============================] - 133s 117ms/step - loss: 3.3599 - categorical_accuracy: 0.0356 - val_loss: 3.3462 - val_categorical_accuracy: 0.0431
Epoch 6/10
 582/1134 [==============>...............] - ETA: 1:02 - loss: 3.3569 - categorical_accuracy: 0.0390

KeyboardInterrupt: 

In [ ]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
directorytest = '/content/testimage.jpg'

image = Image.open(directorytest)
image = image.resize((98, 98))
image = image.convert('L')
image_array = np.array(image)
image_array = image_array/255
image_array = image_array[np.newaxis,:, :, np.newaxis]

realX = image_array
realY = [0, 0, 0 , 0 , 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
realY = np.array(realY)
realY = realY[np.newaxis,:]
print(realY.shape)
score = model.evaluate(realX, realY, verbose=0)

score = model.predict(realX)
print(score)
X_batch = image_array[0]
X_image = X_batch[:, :, 0]
print(X_image.shape)
X_image = X_image*255
X_image = X_image.astype('uint8')
print(X_image)
image = Image.fromarray(X_image, 'L')
image.save
image.show()

plt.imshow(X_image, cmap='gray')